In [1]:
import numpy as np
import warnings
from pyomo.environ import *
from pyomo.opt import SolverFactory
%matplotlib inline

warnings.simplefilter('ignore')

In [2]:
antenna_location = {
    "x": 1,
    "y": 1.5,
}
tag_population = 5
tag_numbers = [i + 1 for i in range(tag_population)]
tags_space = 0.05
frequency = 920.4e6
light_speed = 2.998e8
wave_length = light_speed / frequency
angular_velocity = 2 * np.pi * frequency
sample_count = 1000

# 混合整数非線形計画法


## Sample Data 生成

In [3]:
tags_location = [(m-1)*tags_space for m in tag_numbers]
R = [(antenna_location["x"]-tags_location[m-1])**2 + antenna_location["y"]**2 for m in tag_numbers]
phases = [(r * 4*np.pi / wave_length) % (2*np.pi) for r in R]

In [4]:
def objective_rule1(model):
    return sum(((model.x-(m-1)*tags_space)**2+model.y**2-wave_length/4/np.pi*(phases[m-1]+2*np.pi*model.n[m]))**2 for m in model.M)

def n_initialize(model, i):
    return 1

def constraint_rule2(model, i):
    model.n[i] >= 0

def constraint_rule3(model, i):
    return abs(wave_length/4/np.pi*(phases[i-1]+2*np.pi*model.n[i]-phases[0]-2*np.pi*model.n[1])) <= i*tags_space - 0.000001

def constraint_rule4(model, i):
    return wave_length/4/np.pi*(phases[i-1]+2*np.pi*model.n[i]+phases[0]+2*np.pi*model.n[1]) >= i*tags_space + 0.000001

In [5]:
model = ConcreteModel(name="Nonconvex MINLP sample")
model.x = Var(within=Reals, initialize=1)
model.y = Var(within=Reals, bounds=(0, None), initialize=0)
model.M = Set(initialize=range(1,tag_population+1))
model.n = Var(model.M, domain=Integers, initialize=n_initialize, bounds=(0, None))

model.obj = Objective(rule=objective_rule1, sense=minimize)
# model.constraint1 = Constraint()
# model.constraint2 = Constraint(expr=model.y >= 0)
model.constraint3 = Constraint(model.M, rule=constraint_rule3)
model.constraint4 = Constraint(model.M, rule=constraint_rule4)

for c in model.component_objects():
    c.pprint()

opt = SolverFactory('ipopt')

res = opt.solve(model)

x : Size=1, Index=None
    Key  : Lower : Value : Upper : Fixed : Stale : Domain
    None :  None :     1 :  None : False : False :  Reals
y : Size=1, Index=None
    Key  : Lower : Value : Upper : Fixed : Stale : Domain
    None :     0 :     0 :  None : False : False :  Reals
M : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
n : Size=5, Index=M
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :     1 :  None : False : False : Integers
      2 :     0 :     1 :  None : False : False : Integers
      3 :     0 :     1 :  None : False : False : Integers
      4 :     0 :     1 :  None : False : False : Integers
      5 :     0 :     1 :  None : False : False : Integers
obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : (x**2 + y**2 - 0.025920606224983826*(6.002349139347011 + 6.283185307179586*n[1]))**2 + ((x - 0.05)**2 +

In [6]:
print('optimum value = ', model.obj())
print("x = ", model.x[:]())
print("y = ", model.y[:]())
print("N = ", model.n[:]())

optimum value =  5.270676993716153e-06
x =  [0.08353250829679228]
y =  [41.828595540605015]
N =  [10741.984458450286, 10742.54983227877, 10742.104753437108, 10742.668135602502, 10742.23124205864]
